## Data Exploration
This notebook will allow us to explore the data found in the Kaggle dataset

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

**Kaggle 45k dataset**  
https://www.kaggle.com/datasets/imoore/60k-stack-overflow-questions-with-quality-rate  

In [3]:
df = pd.read_csv('datasets/kaggle-stackoverflow-45k/train.csv')
df.shape

(45000, 6)

In [72]:
df.head(3)

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ


In [73]:
subset = df.iloc[:100]
subset.head(3)

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ


In [6]:
print(subset.iloc[0].Body)

<p>I'm already familiar with repeating tasks every n seconds by using Java.util.Timer and Java.util.TimerTask. But lets say I want to print "Hello World" to the console every random seconds from 1-5. Unfortunately I'm in a bit of a rush and don't have any code to show so far. Any help would be apriciated.  </p>



In the first 100 data points, all tags can be extracted cleanly (i.e. no extra < or > tags)

In [7]:
tagTest = subset.copy()
tagTest = tagTest.assign(Tags = tagTest["Tags"].str.findall(r"<(.*?)>"))
">" in "".join(tagTest["Tags"].sum()) or "<" in "".join(tagTest["Tags"].sum())

False

In [74]:
tagTest.head(3)

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,"[java, repeat]",2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,"[java, optional]",2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,"[javascript, image, overlay, react-native, opa...",2016-01-01 02:48:24,HQ


Associated Links may be found in anchor tags
Different languages may be found in body tags (e.g. stdio.h for C)
This query may be picking up code references that are not necessarily HTML (e.g. <Attraction*>, <?>)

In [9]:
bodyTest = subset.copy()
t = set(bodyTest["Body"].str.findall(r"<[^/].*?>").sum())
for _ in list(t)[:5]:
    print(_)

<code>
<img src="https://i.stack.imgur.com/Me9kV.png" alt="enter image description here">
<!-- News Slider -->
<a href="http://talesofcpp.fusionfenix.com/post-23/interlude" rel="noreferrer">
<em>


Regex will work if there isn't any nesting and the html tag has a separate closing tag.  
For nesting the function can be recursively called until there is no nesting.  
I am considering ignoring images as we will need to deal with image processessing for (presumably) many file formats.

In [10]:
bodyTest["Body"].str.findall(r"<(.*?)>(.*?)<(/\1)>").iloc[2]

[('p',
  'I am attempting to overlay a title over an image - with the image darkened with a lower opacity. However, the opacity effect is changing the overlaying text as well - making it dim. Any fix to this? Here is what is looks like:',
  '/p'),
 ('p',
  '<a href="https://i.stack.imgur.com/1HzD7.png" rel="noreferrer"><img src="https://i.stack.imgur.com/1HzD7.png" alt="enter image description here"></a>',
  '/p'),
 ('p',
  'And here is my code for the custom component (article preview - which the above image is a row of article preview components): ',
  '/p')]

**Cornell Dataset**  
https://www.cs.cornell.edu/~arb/data/stackoverflow-answers/  
Based on the text file contents, the graph approach doesn't have any actual questions/answers immediately available.  
The Cornell dataset might be useful for similarity (although the inference strength is weak with only tags available)

**Kaggle 10% Dataset**  
https://www.kaggle.com/datasets/stackoverflow/stacksample?resource=download  
Need to load using "latin-1" encoding since UTF-8 errors


In [11]:
b10 = Path("datasets") / "kaggle-stackoverflow-10-percent"

In [12]:
ten_percent_questions = pd.read_csv(b10 / "Questions.csv", encoding="latin-1")
ten_percent_questions.head(3)

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...


In [13]:
ten_percent_answers = pd.read_csv(b10 / "Answers.csv", encoding="latin-1")
ten_percent_answers.head(3)

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...


In [14]:
ten_percent_tags = pd.read_csv(b10 / "Tags.csv", encoding="latin-1")
ten_percent_tags.head(3)

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air


In [83]:
tpt = ten_percent_tags.iloc[:100]
tpt = tpt.groupby("Id")["Tag"].apply(list).to_frame()
tpt.head(3)

,Tag
Id,
80,"[flex, actionscript-3, air]"
90,"[svn, tortoisesvn, branch, branching-and-merging]"
120,"[sql, asp.net, sitemap]"


In [66]:
tpq = ten_percent_questions.iloc[:1000]
tpa = ten_percent_answers.iloc[:1000]
tpt = ten_percent_tags.iloc[:1000]

In [71]:
qna = ten_percent_questions.merge(ten_percent_answers, left_on="Id", right_on="ParentId", suffixes=("_question", "_answer"))
qna.head(3)

,Id_question,OwnerUserId_question,CreationDate_question,ClosedDate,Score_question,Title,Body_question,Id_answer,OwnerUserId_answer,CreationDate_answer,ParentId,Score_answer,Body_answer
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
1,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,10008,1109.0,2008-08-13T16:09:09Z,80,6,"<p>The <a href=""http://en.wikipedia.org/wiki/S..."
2,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,3770976,364174.0,2010-09-22T15:37:30Z,80,1,<p>What about making your delimiter something ...


In [ ]:
print(qna.iloc[0].Body_question)

<p>I've written a database generation script in <a href="http://en.wikipedia.org/wiki/SQL">SQL</a> and want to execute it in my <a href="http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime">Adobe AIR</a> application:</p>

<pre><code>Create Table tRole (
      roleID integer Primary Key
      ,roleName varchar(40)
);
Create Table tFile (
    fileID integer Primary Key
    ,fileName varchar(50)
    ,fileDescription varchar(500)
    ,thumbnailID integer
    ,fileFormatID integer
    ,categoryID integer
    ,isFavorite boolean
    ,dateAdded date
    ,globalAccessCount integer
    ,lastAccessTime date
    ,downloadComplete boolean
    ,isNew boolean
    ,isSpotlight boolean
    ,duration varchar(30)
);
Create Table tCategory (
    categoryID integer Primary Key
    ,categoryName varchar(50)
    ,parent_categoryID integer
);
...
</code></pre>

<p>I execute this in Adobe AIR using the following methods:</p>

<pre><code>public static function RunSqlFromFile(fileName:String):void {
    var f

In [70]:
print(qna.iloc[0].Body_answer)

<p>I wound up using this. It is a kind of a hack, but it actually works pretty well. The only thing is you have to be very careful with your semicolons. : D</p>

<pre><code>var strSql:String = stream.readUTFBytes(stream.bytesAvailable);      
var i:Number = 0;
var strSqlSplit:Array = strSql.split(";");
for (i = 0; i &lt; strSqlSplit.length; i++){
    NonQuery(strSqlSplit[i].toString());
}
</code></pre>



We can ignore \<pre\> tags since we won't display the code itself  
We can discuss how to use code snippets (if at all) or if we would prefer to just use generally guidance (i.e. normal sentences)  
We may want to have some sort of reference to the content within \<code\> blocks to extend tag information for each topics

**Applying Changes for Export**  

In [84]:
ten_percent_tags = ten_percent_tags.groupby("Id")["Tag"].apply(list).to_frame()
qna = ten_percent_questions.merge(ten_percent_answers, left_on="Id", right_on="ParentId", suffixes=("_question", "_answer"))
qna = qna.merge(ten_percent_tags, left_on="Id_question", right_index=True)

We could consider combining all the answers for a single question to give a better overall answers, although there should be some indication of differentiation since different parts of the answer may be highlighted.  

In [86]:
qna.head(3)

,Id_question,OwnerUserId_question,CreationDate_question,ClosedDate,Score_question,Title,Body_question,Id_answer,OwnerUserId_answer,CreationDate_answer,ParentId,Score_answer,Body_answer,Tag
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...,"[flex, actionscript-3, air]"
1,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,10008,1109.0,2008-08-13T16:09:09Z,80,6,"<p>The <a href=""http://en.wikipedia.org/wiki/S...","[flex, actionscript-3, air]"
2,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,3770976,364174.0,2010-09-22T15:37:30Z,80,1,<p>What about making your delimiter something ...,"[flex, actionscript-3, air]"


All written content seems to be in \<p\> tags.

In [90]:
qna = qna.assign(question_text_content = qna["Body_question"].str.findall(r"<p>(.*?)</p>"))
qna = qna.assign(answer_text_content = qna["Body_answer"].str.findall(r"<p>(.*?)</p>"))
qna.head(3)

,Id_question,OwnerUserId_question,CreationDate_question,ClosedDate,Score_question,Title,Body_question,Id_answer,OwnerUserId_answer,CreationDate_answer,ParentId,Score_answer,Body_answer,Tag,answer_text_content,question_text_content
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...,"[flex, actionscript-3, air]",[I wound up using this. It is a kind of a hack...,[I've written a database generation script in ...
1,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,10008,1109.0,2008-08-13T16:09:09Z,80,6,"<p>The <a href=""http://en.wikipedia.org/wiki/S...","[flex, actionscript-3, air]","[The <a href=""http://en.wikipedia.org/wiki/SQL...",[I've written a database generation script in ...
2,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,3770976,364174.0,2010-09-22T15:37:30Z,80,1,<p>What about making your delimiter something ...,"[flex, actionscript-3, air]",[What about making your delimiter something a ...,[I've written a database generation script in ...


From here we still need to deal with nested elements (e.g. links, styling, or any other nested tags).  
We may want to include the domain name in links as a reference, but not train predictions on the domain name itself.  
  
Something else to consider is how to deal with placeholders (e.g. SQLStatement.execute() in the titles above). These maybe shouldn't be in the training corpus because they are mostly arbitrary, but also we could potentially teach the model to understand arbitrary items to sometimes be placeholders.
  
We can definitely preprocess the text a fair amount, I just want to be careful that the capitalization, for instance, isn't important (e.g. in class definitions).  
  
Since time stamps are available, we could use the response time as a measure of answer confidence (although this might be troublesome as old posts could be updated to reflect version changes).

In [91]:
qna.isna().sum()

Id_question                    0
OwnerUserId_question       31893
CreationDate_question          0
ClosedDate               1915448
Score_question                 0
Title                          0
Body_question                  0
Id_answer                      0
OwnerUserId_answer         13200
CreationDate_answer            0
ParentId                       0
Score_answer                   0
Body_answer                    0
Tag                            0
answer_text_content            0
question_text_content          0
dtype: int64

The missing values seem to be with Ids and not text so it may well be that we can ignore these missing values.  
We can definitely ignore closed date values (maybe we binarize this since it says whether a post is closed or not).

In [ ]:
# qna.to_csv("./datasets/kaggle-stackoverflow-10-percent/combined_ten_percent_qna.csv")